Adding Faces for Dlib

In [1]:
import cv2
import os
import face_recognition
import pickle

FOLDER = "known_faces_dlib"
ENCODINGS_FILE = "known_faces_dlib.pkl"

# Ensure main folder exists
os.makedirs(FOLDER, exist_ok=True)

# Ask for the person's name
name = input("👤 Enter the person's name (no spaces): ").strip()
if not name:
    print("❌ Invalid name.")
    exit()

# Create person's subfolder
person_folder = os.path.join(FOLDER, name)
os.makedirs(person_folder, exist_ok=True)

# Load encodings if available and make sure it's a dict
known_faces = {}
if os.path.exists(ENCODINGS_FILE):
    with open(ENCODINGS_FILE, "rb") as f:
        data = pickle.load(f)
        if isinstance(data, dict):
            known_faces = data
        else:
            print("⚠️ Invalid encoding format detected. Resetting known_faces.pkl.")
            os.remove(ENCODINGS_FILE)
            known_faces = {}

# Ensure name exists in dict
if name not in known_faces:
    known_faces[name] = []

# Start webcam
cap = cv2.VideoCapture(0)
print("📸 Press SPACE to capture a face image. Press ESC when done.")

img_count = 1

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read from webcam.")
        break

    cv2.imshow("New Face Capture", frame)
    key = cv2.waitKey(1)

    if key % 256 == 27:  # ESC
        print("✅ Done capturing images.")
        break
    elif key % 256 == 32:  # SPACE
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        encodings = face_recognition.face_encodings(rgb)

        if not encodings:
            print("❌ No face found. Try again.")
            continue

        # Save image
        save_path = os.path.join(person_folder, f"{img_count}.jpg")
        cv2.imwrite(save_path, frame)
        print(f"✅ Image {img_count} saved as {save_path}")

        # Save encoding
        known_faces[name].append(encodings[0])
        img_count += 1

cap.release()
cv2.destroyAllWindows()

# Save encodings to file
with open(ENCODINGS_FILE, "wb") as f:
    pickle.dump(known_faces, f)

print("✅ Encodings saved to known_faces.pkl.")


📸 Press SPACE to capture a face image. Press ESC when done.
✅ Image 1 saved as known_faces_dlib/EfeKaanEfe/1.jpg
✅ Image 2 saved as known_faces_dlib/EfeKaanEfe/2.jpg
✅ Image 3 saved as known_faces_dlib/EfeKaanEfe/3.jpg
❌ No face found. Try again.
✅ Image 4 saved as known_faces_dlib/EfeKaanEfe/4.jpg
✅ Done capturing images.
✅ Encodings saved to known_faces.pkl.


Adding Faces for Arcface

In [2]:
import cv2
import os
import pickle
import numpy as np
import insightface

FOLDER = "known_faces_arcface"
ENCODINGS_FILE = "known_faces_arcface.pkl"

# Ensure folder exists
os.makedirs(FOLDER, exist_ok=True)

# Ask for the person's name
name = input("👤 Enter the person's name (no spaces): ").strip()
if not name:
    print("❌ Invalid name.")
    exit()

# Create person's folder
person_folder = os.path.join(FOLDER, name)
os.makedirs(person_folder, exist_ok=True)

# Load or initialize known faces
known_faces = {}
if os.path.exists(ENCODINGS_FILE):
    with open(ENCODINGS_FILE, "rb") as f:
        data = pickle.load(f)
        if isinstance(data, dict):
            known_faces = data
        else:
            print("⚠️ Invalid encoding format detected. Resetting.")
            os.remove(ENCODINGS_FILE)
            known_faces = {}

# Ensure person entry exists
if name not in known_faces:
    known_faces[name] = []

# Prepare ArcFace model
model = insightface.app.FaceAnalysis(name='buffalo_l')
model.prepare(ctx_id=0)

# Webcam capture loop
cap = cv2.VideoCapture(0)
print("📸 Press SPACE to capture a face image. Press ESC when done.")

img_count = 1

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Webcam read failed.")
        break

    # Detect face and draw boxes (optional)
    faces = model.get(frame)
    for face in faces:
        box = face.bbox.astype(int)
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

    cv2.imshow("ArcFace Capture", frame)
    key = cv2.waitKey(1)

    if key % 256 == 27:  # ESC
        print("✅ Done capturing.")
        break
    elif key % 256 == 32:  # SPACE
        faces = model.get(frame)
        if not faces:
            print("❌ No face detected. Try again.")
            continue

        embedding = faces[0].embedding
        embedding = embedding / np.linalg.norm(embedding)  # Normalize

        # Save image
        save_path = os.path.join(person_folder, f"{img_count}.jpg")
        cv2.imwrite(save_path, frame)
        print(f"✅ Saved {save_path}")

        # Save embedding
        known_faces[name].append(embedding)
        img_count += 1

cap.release()
cv2.destroyAllWindows()

# Save updated encodings
with open(ENCODINGS_FILE, "wb") as f:
    pickle.dump(known_faces, f)

print(f"✅ All encodings saved to {ENCODINGS_FILE}")


/home/maksimgorki/.local/lib/python3.12/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/maksimgorki/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/maksimgorki/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/maksimgorki/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/maksimgorki/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/maksimgorki/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3,

/home/maksimgorki/.local/lib/python3.12/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


✅ Done capturing.
✅ All encodings saved to known_faces_arcface.pkl


Adding Faces for Deepface

In [ ]:
import cv2
import os
import pickle
from deepface import DeepFace

# === Settings ===
FOLDER = "known_faces_deepface"
ENCODING_FILE = "known_faces_deepface.pkl"
MODEL_NAME = "ArcFace"

# === Ask for person name ===
name = input("👤 Enter the person's name (no spaces): ").strip()
if not name:
    print("❌ Invalid name.")
    exit()

# === Create person's folder ===
person_folder = os.path.join(FOLDER, name)
os.makedirs(person_folder, exist_ok=True)

# === Capture from webcam ===
cap = cv2.VideoCapture(0)
print("📸 Press SPACE to capture a face image. Press ESC when done.")

img_count = 1
captured_paths = []

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Webcam read failed.")
        break

    cv2.imshow("Capture Face", frame)
    key = cv2.waitKey(1)

    if key % 256 == 27:  # ESC
        print("✅ Done capturing.")
        break

    elif key % 256 == 32:  # SPACE
        save_path = os.path.join(person_folder, f"{img_count}.jpg")
        cv2.imwrite(save_path, frame)
        captured_paths.append(save_path)
        print(f"✅ Saved image {img_count} at {save_path}")
        img_count += 1

cap.release()
cv2.destroyAllWindows()

if not captured_paths:
    print("❗ No images captured.")
    exit()

# === Load or initialize database ===
if os.path.exists(ENCODING_FILE):
    with open(ENCODING_FILE, "rb") as f:
        database = pickle.load(f)
else:
    database = {}

database[name] = []

# === Encode captured photos ===
print(f"🧠 Encoding {len(captured_paths)} image(s) for {name} using {MODEL_NAME}...")

for img_path in captured_paths:
    try:
        reps = DeepFace.represent(
            img_path=img_path,
            model_name=MODEL_NAME,
            enforce_detection=False,
        )
        if reps and isinstance(reps, list):
            emb = reps[0]["embedding"]
            database[name].append(emb)
            print(f"[+] Encoded: {os.path.basename(img_path)}")
        else:
            print(f"[!] No embedding for {img_path}")
    except Exception as e:
        print(f"[!] Error processing {img_path}: {e}")

# === Save to file ===
with open(ENCODING_FILE, "wb") as f:
    pickle.dump(database, f)

print(f"\n✅ Updated embeddings saved to {ENCODING_FILE}")
